In [1]:
import pandas as pd

In [2]:
bookings = pd.read_csv('2_bookings.csv', sep=';')

In [5]:
#первые 7 строк датасета
bookings.head(7)

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


In [6]:
#размер датасета
bookings.shape

(119390, 21)

In [11]:
#к какому типу относится большинство переменных? 
bookings.dtypes.value_counts()

object     10
int64      10
float64     1
dtype: int64

In [25]:
#замена пробелов и верхнего регистра в названиях столбцов
def replace_name(name):
    new_name = name.replace(' ', '_').lower()
    return new_name

bookings.rename(columns=replace_name, inplace=True)

In [31]:
#топ-5 стран с наибольшим числом успешных бронирований (is_canceled = 0)
bookings.query('is_canceled == 0') \
        .groupby('country', as_index=False) \
        .agg({'hotel': 'count'}) \
        .rename(columns={'hotel': 'successful_bookings'}) \
        .sort_values('successful_bookings', ascending=False).head()

,country,successful_bookings
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069


In [34]:
#среднее количество ночей в отелях разных типов
bookings.groupby('hotel', as_index=False) \
        .agg({'stays_total_nights': 'mean'}).round(2)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


In [42]:
#количество раз, когда тип номера отличался от изначально забронированного
difference = pd.Series(bookings.assigned_room_type != bookings.reserved_room_type)
difference.sum()

14917

In [49]:
#самые популярные месяцы в 2016 и 2017 годах
bookings.query('arrival_date_year == 2016 or arrival_date_year == 2017') \
        .groupby(['arrival_date_year', 'arrival_date_month'], as_index=False) \
        .agg({'hotel': 'count'}) \
        .rename(columns={'hotel': 'bookings'}) \
        .sort_values('bookings', ascending=False)

,arrival_date_year,arrival_date_month,bookings
19,2017,May,6313
10,2016,October,6203
12,2017,April,5661
17,2017,June,5647
8,2016,May,5478
0,2016,April,5428
11,2016,September,5394
16,2017,July,5313
6,2016,June,5292
1,2016,August,5063


In [52]:
#рекордные по отменам месяцы в разные годы
bookings.query('hotel == "City Hotel" and is_canceled == 1') \
        .groupby('arrival_date_year', as_index=False)['arrival_date_month'] \
        .value_counts().sort_values('count', ascending=False)

,arrival_date_year,arrival_date_month,count
18,2017,May,2217
6,2016,October,1947
19,2017,April,1926
20,2017,June,1808
7,2016,June,1720
8,2016,September,1567
0,2015,September,1543
9,2016,April,1539
10,2016,May,1436
11,2016,November,1360


In [57]:
#усредненный портрет посетителей (включая отмены)
bookings[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

In [60]:
#среднее количество детей у клиентов в разных типах отелей
bookings['total_kids'] = bookings.children + bookings.babies

bookings.groupby('hotel', as_index=False) \
        .agg({'total_kids': 'mean'}).round(2)

,hotel,total_kids
0,City Hotel,0.10
1,Resort Hotel,0.14


#### Churn Rate для клиентов с детьми и без

In [87]:
#создадим столбец с информацией, есть ли у клиента дети
bookings['has_kids'] = bookings.total_kids != 0

In [105]:
#общее число клиентов с детьми
overall_with_kids = bookings.query('has_kids == True').shape[0]
#число клиентов с детьми и отменивших бронирование
canceled_with_kids = bookings.query('has_kids == True and is_canceled == 1').shape[0]

#процент клиентов c детьми, отменивших бронирование
churn_rate_with_kids = canceled_with_kids / overall_with_kids * 100
round(churn_rate_with_kids, 2)

34.95

In [106]:
#общее число клиентов без детей
overall_without_kids = bookings.query('has_kids == False').shape[0]
#число клиентов без детей и отменивших бронирование
canceled_without_kids = bookings.query('has_kids == False and is_canceled == 1').shape[0]

#процент клиентов без детей, отменивших бронирование
churn_rate_without_kids = canceled_without_kids / overall_without_kids * 100
round(churn_rate_without_kids, 2)

37.22